In [1]:
import os
os.chdir('../../modules/')

In [14]:
import re
import pickle
from util.util import *
from util.losses import *
from util.pred_blocks import ConvEncoder, GRUDecoder
from likelihood_predictor import PlastPredictor
from vae_generator import PlastVAEGen
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [4]:
%load_ext autoreload
%autoreload 2

import IPython.display as Disp
np.set_printoptions(suppress=True)

In [5]:
pl_ll = pd.read_pickle('../database/pl_likelihoods_v1.pkl')
org_ll = pd.read_pickle('../database/org_likelihoods_v1.pkl')
gdb_data = pd.read_pickle('../database/gdb_vae_test_1mil.pkl').to_numpy()
all_data = pd.concat([pl_ll, org_ll]).to_numpy()
test_data = pd.concat([pl_ll, org_ll]).sample(n=500).to_numpy()

In [7]:
max_len = 100
pvg = PlastVAEGen(params={'MAX_LENGTH': max_len, 'N_EPOCHS': 10, 'BATCH_SIZE': 10}, weigh_freq=False)
pvg.initiate(pl_ll.to_numpy())

In [8]:
pvg.params['CHAR_DICT']

{'C': 0,
 '(': 1,
 ')': 2,
 'O': 3,
 '=': 4,
 '1': 5,
 '2': 6,
 'S': 7,
 '[O-]': 8,
 '3': 9,
 '4': 10,
 '.': 11,
 '[Na+]': 12,
 'P': 13,
 '/': 14,
 '\\': 15,
 'N': 16,
 'Br': 17,
 '[Zn+2]': 18,
 'Cl': 19,
 '5': 20,
 '[N+]': 21,
 '[C@]': 22,
 '[C@@H]': 23,
 '[C@H]': 24,
 '_': 25}

In [9]:
max_len = 45
pvg = PlastVAEGen(params={'MAX_LENGTH': max_len, 'N_EPOCHS': 10, 'BATCH_SIZE': 10})
pvg.initiate(gdb_data)

In [10]:
pvg.params['CHAR_DICT']

{'Br': 0,
 'C': 1,
 '1': 2,
 '=': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 'N': 7,
 '(': 8,
 '#': 9,
 ')': 10,
 'O': 11,
 'S': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 'c': 16,
 's': 17,
 'n': 18,
 '[nH]': 19,
 'o': 20,
 'Cl': 21,
 'F': 22,
 'I': 23,
 '[N+]': 24,
 '[O-]': 25,
 '_': 26}

Two tasks left for input data. 
1. Need to add weighting function to `initiate()` so that more frequent characters have a smaller effect on the loss and the model will learn the less frequent characters more effectively.
2. Need to regularize the incoming tokens for the PL, ORG and GDB datasets.

In [13]:
bad_chars = []
for k, v in pvg.char_w.items():
    if v < 100:
        bad_chars.append(k)

AttributeError: 'PlastVAEGen' object has no attribute 'char_w'

In [ ]:
good_idxs = []
all_gdb_smiles = gdb_data[:,0]
for i, smile in enumerate(all_gdb_smiles):
    bad_smile = False
    for bad_char in bad_chars:
        if bad_char in smile:
            bad_smile = True
    if bad_smile:
        pass
    else:
        good_idxs.append(i)

In [ ]:
gdb_data = gdb_data[good_idxs,:]

In [ ]:
len(gdb_data)

Determining like characters between GDB, PL and ORG

In [17]:
with open('../scripts/gdb_char_dict.pkl', 'rb') as f:
    gdb_char_dict = pickle.load(f)

In [20]:
total_chars = 0
for k, v in gdb_char_dict.items():
    total_chars += v

pop_items = []
for k, v in gdb_char_dict.items():
    perc = round(v / total_chars * 100, 5)
    if perc < 0.01:
        pop_items.append(k)

In [21]:
for item in pop_items:
    gdb_char_dict.pop(item)

In [25]:
gdb_char_dict

{'Br': 1179939,
 'C': 578495733,
 '1': 120088320,
 '=': 114378468,
 '2': 68830550,
 '3': 16223832,
 '4': 2640190,
 'N': 129434138,
 '(': 158583035,
 '#': 16752553,
 ')': 158583035,
 'O': 105517366,
 'S': 8506426,
 '5': 271622,
 'F': 1194144,
 'Cl': 464279,
 'I': 427590}

In [23]:
pop_items

['c', 'n', '[nH]', 's', 'o', '6', '7', '8', '[N+]', '[O-]']